In [1]:
from collections import deque
import model
from dataUtils import *
from logger import MyLogger
import sys
import PTB_data_reader
import time
import numpy as np
import lstm_char_cnn
import config
import pickle
import dataloader

tf.logging.set_verbosity(tf.logging.ERROR)

logger = MyLogger("ERDMain")

W0827 14:47:14.211623 140154599094080 deprecation_wrapper.py:119] From /home/hadoop/ERD/model.py:6: The name tf.losses.Reduction is deprecated. Please use tf.compat.v1.losses.Reduction instead.

Using TensorFlow backend.


In [2]:
tf.app.flags.DEFINE_string('f', '', 'kernel')
 #load PTB data
# word_vocab, char_vocab, word_tensors, char_tensors, max_word_length = \
#     PTB_data_reader.load_data(FLAGS.data_dir, FLAGS.max_word_length, char_vocab, eos=FLAGS.EOS)
word_vocab, char_vocab, word_tensors, char_tensors = \
    PTB_data_reader.load_data_fast()
max_word_length = FLAGS.max_word_length
train_reader = PTB_data_reader.DataReader(word_tensors['train'], char_tensors['train'],
                          FLAGS.batch_size, FLAGS.num_unroll_steps) 

#load sentiment analysis data
sentiReader = dataloader.SentiDataLoader(
                                        dirpath = '/home/hadoop/trainingandtestdata',
                                        trainfile = 'training.1600000.processed.noemoticon.csv', 
                                        testfile = 'testdata.manual.2009.06.14.csv', 
                                        charVocab = char_vocab
                        )
# sentiReader.load_data()
sentiReader.load_data_fast(
                        '/home/hadoop/ERD/data/senti_train_data.pickle',
                        '/home/hadoop/ERD/data/senti_train_label.pickle',
                        '/home/hadoop/ERD/data/senti_test_data.pickle',
                        '/home/hadoop/ERD/data/senti_test_label.pickle'
                          )

# load twitter data
# load_data(FLAGS.data_file_path)
load_data_fast()

# (self, input_dim, hidden_dim, max_seq_len, max_word_num, class_num, action_num):
print(  FLAGS.embedding_dim, FLAGS.hidden_dim, 
            FLAGS.max_seq_len, FLAGS.max_sent_len, 
                FLAGS.class_num, FLAGS.action_num   )
logger.info(    (FLAGS.embedding_dim, FLAGS.hidden_dim, 
                    FLAGS.max_seq_len, FLAGS.max_sent_len, 
                        FLAGS.class_num, FLAGS.action_num)  )

print(get_curtime() + " Data loaded.")
logger.info(get_curtime() + " Data loaded.")

I0827 14:47:32.684508 140154599094080 logger.py:24] (300, 200, 101, 31, 2, 2)
I0827 14:47:32.685089 140154599094080 logger.py:24] 2019-08-27 14:47:32 Data loaded.


max_sent: 31 ,  max_seq_len: 101
5802 data loaded
300 200 101 31 2 2
2019-08-27 14:47:32 Data loaded.


In [3]:
# # save the Twitter data
# data = get_data()
# with open('data/data_dict.txt', 'wb') as handle:
#     pickle.dump(data, handle, protocol=pickle.HIGHEST_PROTOCOL)

# # save the PTB data
# with open('data/char_tensors.txt', 'wb') as handle:
#     pickle.dump(char_tensors, handle, protocol=pickle.HIGHEST_PROTOCOL)

# with open('data/word_tensors.txt', 'wb') as handle:
#     pickle.dump(word_tensors, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
# with open('data/char_vocab.txt', 'wb') as handle:
#     pickle.dump(char_vocab, handle, protocol=pickle.HIGHEST_PROTOCOL)

# with open('data/word_vocab.txt', 'wb') as handle:
#     pickle.dump(word_vocab, handle, protocol=pickle.HIGHEST_PROTOCOL)

# save the senti data
# with open('data/senti_train_data.pickle', 'wb') as handle:
#     pickle.dump(sentiReader.train_data, handle, protocol=pickle.HIGHEST_PROTOCOL)

# with open('data/senti_train_label.pickle', 'wb') as handle:
#     pickle.dump(sentiReader.train_label, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
# with open('data/senti_test_data.pickle', 'wb') as handle:
#     pickle.dump(sentiReader.test_data, handle, protocol=pickle.HIGHEST_PROTOCOL)

# with open('data/senti_test_label.pickle', 'wb') as handle:
#     pickle.dump(sentiReader.test_label, handle, protocol=pickle.HIGHEST_PROTOCOL)



In [4]:
w2v = lstm_char_cnn.WordEmbedding(
                max_word_length = FLAGS.max_char_num , 
                char_vocab_size = char_vocab.size, 
                char_embed_size = FLAGS.char_embed_size, 
                kernels = eval(FLAGS.kernels), 
                kernel_features = eval(FLAGS.kernel_features), 
                num_highway_layers = FLAGS.highway_layers,
                embedding_dim = FLAGS.embedding_dim
            )
lstm_lm = lstm_char_cnn.LSTM_LM(
            batch_size = FLAGS.batch_size, 
            num_unroll_steps = FLAGS.num_unroll_steps, 
            rnn_size = FLAGS.rnn_size, 
            num_rnn_layers = FLAGS.rnn_layers, 
            word_vocab_size = word_vocab.size
        )

char_train_graph = lstm_char_cnn.infer_train_model(
                    w2v, lstm_lm, 
                    batch_size = FLAGS.batch_size, 
                    num_unroll_steps = FLAGS.num_unroll_steps, 
                    max_word_length = FLAGS.max_char_num, 
                    learning_rate = FLAGS.learning_rate,
                    max_grad_norm = FLAGS.max_grad_norm
                 )
#sentiment analysis model
s_model = model.SentiModel(FLAGS.hidden_dim, 5)
senti_train_graph = model.InferSentiTrainGraph(
                        w2v, 
                        s_model, 
                        max_word_num = FLAGS.max_sent_len, 
                        max_char_num = FLAGS.max_char_num, 
                        hidden_dim = FLAGS.hidden_dim, 
                        sent_num = FLAGS.sent_num,
                        embedding_dim = FLAGS.embedding_dim
                    )
# df model
rdm_model = model.RDM_Model(
                max_seq_len = FLAGS.max_seq_len, 
                max_word_num = FLAGS.max_sent_len, 
                embedding_dim = FLAGS.embedding_dim, 
                hidden_dim = FLAGS.hidden_dim
            )
rdm_train_graph = model.InferRDMTrainGraph(
                        w2v, s_model, rdm_model, 
                        max_seq_len = FLAGS.max_seq_len, 
                        max_word_num = FLAGS.max_sent_len, 
                        max_char_num = FLAGS.max_char_num, 
                        hidden_dim = FLAGS.hidden_dim, 
                        embedding_dim = FLAGS.embedding_dim,
                        class_num = FLAGS.class_num
                )

# rl model
cm_model = model.CM_Model(
                    max_word_num = FLAGS.max_sent_len, 
                    embedding_dim = FLAGS.embedding_dim, 
                    hidden_dim = FLAGS.hidden_dim, 
                    action_num = FLAGS.action_num
            )
cm_train_graph = model.InferCMTrainGraph(
                        w2v, s_model, rdm_model, cm_model, 
                        max_word_num = FLAGS.max_sent_len, 
                        embedding_dim = FLAGS.embedding_dim, 
                        hidden_dim = FLAGS.hidden_dim, 
                        action_num = FLAGS.action_num
                    )

saver = tf.train.Saver(tf.global_variables(), max_to_keep=4)
sess = tf.Session()
with sess.as_default():
    sess.run(tf.global_variables_initializer())

summary_writer = tf.summary.FileWriter(FLAGS.train_dir, graph=sess.graph)

input_: Tensor("input:0", shape=(20, 35, 21), dtype=int32)
input_cnn: Tensor("Embedding_1/CNN_OUT/add_7:0", shape=(700, 1100), dtype=float32)
final_embedding Tensor("Embedding_1/dense/Sigmoid:0", shape=(700, 300), dtype=float32)
input_: Tensor("Placeholder_1:0", shape=(?, 31, 21), dtype=int32)
input_cnn: Tensor("Embedding_2/CNN_OUT/add_7:0", shape=(?, 1100), dtype=float32)
final_embedding Tensor("Embedding_2/dense/Sigmoid:0", shape=(?, 300), dtype=float32)
input_: Tensor("Reshape_2:0", shape=(?, 31, 21), dtype=int32)
input_cnn: Tensor("Embedding_3/CNN_OUT/add_7:0", shape=(?, 1100), dtype=float32)
final_embedding Tensor("Embedding_3/dense/Sigmoid:0", shape=(?, 300), dtype=float32)
pooled_rl_input: Tensor("Reshape_5:0", shape=(?, 200), dtype=float32)
rl_state: Tensor("rl_states:0", shape=(?, 200), dtype=float32)


In [7]:
import importlib
importlib.reload(lstm_char_cnn)

<module 'lstm_char_cnn' from '/home/hadoop/ERD/lstm_char_cnn.py'>

In [ ]:
lstm_char_cnn.Train_Char_Model(sess, char_train_graph, train_reader, saver, summary_writer)

    34: 0 [    5/ 1327], train_loss/perplexity = 7.00820255/1105.6652832 secs/batch = 1.3773s, grad.norm=1.10350180
    39: 0 [   10/ 1327], train_loss/perplexity = 6.94740629/1040.4476318 secs/batch = 1.2614s, grad.norm=2.40912032
    44: 0 [   15/ 1327], train_loss/perplexity = 7.09035158/1200.3297119 secs/batch = 1.2492s, grad.norm=0.62322396
    49: 0 [   20/ 1327], train_loss/perplexity = 7.16882467/1298.3177490 secs/batch = 1.2724s, grad.norm=1.75169051
    54: 0 [   25/ 1327], train_loss/perplexity = 6.97885656/1073.6899414 secs/batch = 1.2633s, grad.norm=0.52654147
    59: 0 [   30/ 1327], train_loss/perplexity = 6.75696754/860.0302124 secs/batch = 1.2567s, grad.norm=0.55534643
    64: 0 [   35/ 1327], train_loss/perplexity = 6.96802521/1062.1231689 secs/batch = 1.3029s, grad.norm=0.75841022
    69: 0 [   40/ 1327], train_loss/perplexity = 6.80093956/898.6912842 secs/batch = 1.4808s, grad.norm=0.50612611
    74: 0 [   45/ 1327], train_loss/perplexity = 6.57951593/720.1906128 se

   394: 0 [  365/ 1327], train_loss/perplexity = 6.65772533/778.7774658 secs/batch = 1.2299s, grad.norm=0.33532420
   399: 0 [  370/ 1327], train_loss/perplexity = 6.78186893/881.7150269 secs/batch = 1.2431s, grad.norm=0.51567829
   404: 0 [  375/ 1327], train_loss/perplexity = 6.50124311/665.9689941 secs/batch = 1.2314s, grad.norm=0.37611791
   409: 0 [  380/ 1327], train_loss/perplexity = 6.66318798/783.0432739 secs/batch = 1.2312s, grad.norm=0.43182096
   414: 0 [  385/ 1327], train_loss/perplexity = 6.83460808/929.4639893 secs/batch = 1.2305s, grad.norm=0.48492444
   419: 0 [  390/ 1327], train_loss/perplexity = 6.60802317/741.0167236 secs/batch = 1.2325s, grad.norm=0.35548833
   424: 0 [  395/ 1327], train_loss/perplexity = 6.83409309/928.9854736 secs/batch = 1.2346s, grad.norm=0.38335186
   429: 0 [  400/ 1327], train_loss/perplexity = 6.53138828/686.3504028 secs/batch = 1.2310s, grad.norm=0.70329678
   434: 0 [  405/ 1327], train_loss/perplexity = 6.78411198/883.6950073 secs/bat

   754: 0 [  725/ 1327], train_loss/perplexity = 6.43489265/623.2156982 secs/batch = 1.2393s, grad.norm=0.61607856
   759: 0 [  730/ 1327], train_loss/perplexity = 6.44006634/626.4483643 secs/batch = 1.2326s, grad.norm=0.32190192
   764: 0 [  735/ 1327], train_loss/perplexity = 6.59856844/734.0435791 secs/batch = 1.2255s, grad.norm=0.30069298
   769: 0 [  740/ 1327], train_loss/perplexity = 6.46184969/640.2442017 secs/batch = 1.2343s, grad.norm=0.42999017
   774: 0 [  745/ 1327], train_loss/perplexity = 6.60869551/741.5150757 secs/batch = 1.2357s, grad.norm=0.40313420
   779: 0 [  750/ 1327], train_loss/perplexity = 6.45271969/634.4254150 secs/batch = 1.2383s, grad.norm=0.40159151
   784: 0 [  755/ 1327], train_loss/perplexity = 6.49926567/664.6533813 secs/batch = 1.2403s, grad.norm=0.29774725
   789: 0 [  760/ 1327], train_loss/perplexity = 6.42754793/618.6550903 secs/batch = 1.2359s, grad.norm=0.41735300
   794: 0 [  765/ 1327], train_loss/perplexity = 6.45896769/638.4016724 secs/bat

  1114: 0 [ 1085/ 1327], train_loss/perplexity = 6.51777220/677.0683594 secs/batch = 1.2374s, grad.norm=0.43099278
  1119: 0 [ 1090/ 1327], train_loss/perplexity = 6.59897041/734.3387451 secs/batch = 1.2277s, grad.norm=0.25774765
  1124: 0 [ 1095/ 1327], train_loss/perplexity = 6.57662868/718.1142578 secs/batch = 1.2458s, grad.norm=0.43169427
  1129: 0 [ 1100/ 1327], train_loss/perplexity = 6.64209700/766.7010498 secs/batch = 1.2550s, grad.norm=0.44264206
  1134: 0 [ 1105/ 1327], train_loss/perplexity = 6.46751690/643.8829346 secs/batch = 1.2243s, grad.norm=0.38996848
  1139: 0 [ 1110/ 1327], train_loss/perplexity = 6.77181292/872.8929443 secs/batch = 1.2325s, grad.norm=0.33179608
  1144: 0 [ 1115/ 1327], train_loss/perplexity = 6.52630568/682.8707886 secs/batch = 1.2326s, grad.norm=0.35794267
  1149: 0 [ 1120/ 1327], train_loss/perplexity = 6.50986242/671.7340088 secs/batch = 1.2319s, grad.norm=0.33255973
  1154: 0 [ 1125/ 1327], train_loss/perplexity = 6.71059990/821.0630493 secs/bat

  1471: 1 [  115/ 1327], train_loss/perplexity = 6.32652473/559.2098389 secs/batch = 1.2294s, grad.norm=0.40722099
  1476: 1 [  120/ 1327], train_loss/perplexity = 6.58059645/720.9692383 secs/batch = 1.2343s, grad.norm=0.30056882
  1481: 1 [  125/ 1327], train_loss/perplexity = 6.68163872/797.6251221 secs/batch = 1.2250s, grad.norm=0.28531429
  1486: 1 [  130/ 1327], train_loss/perplexity = 6.63264322/759.4869995 secs/batch = 1.2340s, grad.norm=0.39011109
  1491: 1 [  135/ 1327], train_loss/perplexity = 6.54185390/693.5712280 secs/batch = 1.2427s, grad.norm=0.33109444
  1496: 1 [  140/ 1327], train_loss/perplexity = 6.62974548/757.2894287 secs/batch = 1.2249s, grad.norm=0.26770481
  1501: 1 [  145/ 1327], train_loss/perplexity = 6.72822762/835.6648560 secs/batch = 1.2409s, grad.norm=0.30021721
  1506: 1 [  150/ 1327], train_loss/perplexity = 6.53570032/689.3163452 secs/batch = 1.2301s, grad.norm=0.24829717
  1511: 1 [  155/ 1327], train_loss/perplexity = 6.63650131/762.4228516 secs/bat

...

  3628: 2 [  945/ 1327], train_loss/perplexity = 6.60652685/739.9087524 secs/batch = 1.2288s, grad.norm=0.22155601
  3633: 2 [  950/ 1327], train_loss/perplexity = 6.47243500/647.0573730 secs/batch = 1.2389s, grad.norm=0.24117804
  3638: 2 [  955/ 1327], train_loss/perplexity = 6.61302376/744.7315063 secs/batch = 1.2340s, grad.norm=0.24087793
  3643: 2 [  960/ 1327], train_loss/perplexity = 6.66039848/780.8620605 secs/batch = 1.2438s, grad.norm=0.23963091
  3648: 2 [  965/ 1327], train_loss/perplexity = 6.54907084/698.5947876 secs/batch = 1.2301s, grad.norm=0.24977641
  3653: 2 [  970/ 1327], train_loss/perplexity = 6.58518362/724.2840576 secs/batch = 1.2333s, grad.norm=0.23458621
  3658: 2 [  975/ 1327], train_loss/perplexity = 6.54317379/694.4872437 secs/batch = 1.2354s, grad.norm=0.27399260
  3663: 2 [  980/ 1327], train_loss/perplexity = 6.44207191/627.7059937 secs/batch = 1.2418s, grad.norm=0.23347902
  3668: 2 [  985/ 1327], train_loss/perplexity = 6.61138725/743.5137329 secs/bat

  3988: 2 [ 1305/ 1327], train_loss/perplexity = 6.67845345/795.0885010 secs/batch = 1.2274s, grad.norm=0.26380280
  3993: 2 [ 1310/ 1327], train_loss/perplexity = 6.70523834/816.6726685 secs/batch = 1.2353s, grad.norm=0.23391783
  3998: 2 [ 1315/ 1327], train_loss/perplexity = 6.65894175/779.7253418 secs/batch = 1.2332s, grad.norm=0.20513377
  4003: 2 [ 1320/ 1327], train_loss/perplexity = 6.68859291/803.1912842 secs/batch = 1.2366s, grad.norm=0.22227649
  4008: 2 [ 1325/ 1327], train_loss/perplexity = 6.58517790/724.2799072 secs/batch = 1.2314s, grad.norm=0.23450808
Epoch training time: 1642.2195327281952
Saved char model cv/epoch002_6.5612.model
  4015: 3 [    5/ 1327], train_loss/perplexity = 6.72036886/829.1232910 secs/batch = 1.2305s, grad.norm=0.26981661
  4020: 3 [   10/ 1327], train_loss/perplexity = 6.38104057/590.5418701 secs/batch = 1.2357s, grad.norm=0.23193634
  4025: 3 [   15/ 1327], train_loss/perplexity = 6.39675140/599.8930664 secs/batch = 1.2378s, grad.norm=0.2417306

  4345: 3 [  335/ 1327], train_loss/perplexity = 6.14304399/465.4682922 secs/batch = 1.2244s, grad.norm=0.24833871
  4350: 3 [  340/ 1327], train_loss/perplexity = 6.56716251/711.3485107 secs/batch = 1.2286s, grad.norm=0.23554575
  4355: 3 [  345/ 1327], train_loss/perplexity = 6.60878134/741.5787354 secs/batch = 1.2349s, grad.norm=0.23562001
  4360: 3 [  350/ 1327], train_loss/perplexity = 6.53795671/690.8734741 secs/batch = 1.2482s, grad.norm=0.28343269
  4365: 3 [  355/ 1327], train_loss/perplexity = 6.62040329/750.2476196 secs/batch = 1.2260s, grad.norm=0.23852426
  4370: 3 [  360/ 1327], train_loss/perplexity = 6.68972111/804.0979614 secs/batch = 1.2319s, grad.norm=0.20762087
  4375: 3 [  365/ 1327], train_loss/perplexity = 6.61235619/744.2344971 secs/batch = 1.2300s, grad.norm=0.21111192
  4380: 3 [  370/ 1327], train_loss/perplexity = 6.63595152/762.0037842 secs/batch = 1.2311s, grad.norm=0.29862106
  4385: 3 [  375/ 1327], train_loss/perplexity = 6.46285629/640.8890381 secs/bat

  4705: 3 [  695/ 1327], train_loss/perplexity = 6.54666805/696.9182129 secs/batch = 1.2357s, grad.norm=0.26752213
  4710: 3 [  700/ 1327], train_loss/perplexity = 6.60264921/737.0451660 secs/batch = 1.2531s, grad.norm=0.22076491
  4715: 3 [  705/ 1327], train_loss/perplexity = 6.42459822/616.8329468 secs/batch = 1.2463s, grad.norm=0.27706805
  4720: 3 [  710/ 1327], train_loss/perplexity = 6.55278254/701.1925659 secs/batch = 1.2586s, grad.norm=0.21886456
  4725: 3 [  715/ 1327], train_loss/perplexity = 6.56244898/708.0034790 secs/batch = 1.2790s, grad.norm=0.23184837
  4730: 3 [  720/ 1327], train_loss/perplexity = 6.58110762/721.3378296 secs/batch = 1.2403s, grad.norm=0.26381317
  4735: 3 [  725/ 1327], train_loss/perplexity = 6.38118649/590.6280518 secs/batch = 1.2476s, grad.norm=0.32590616
  4740: 3 [  730/ 1327], train_loss/perplexity = 6.40412664/604.3337402 secs/batch = 1.2312s, grad.norm=0.25269684
  4745: 3 [  735/ 1327], train_loss/perplexity = 6.57075977/713.9120483 secs/bat

  5065: 3 [ 1055/ 1327], train_loss/perplexity = 6.69435358/807.8315430 secs/batch = 1.2451s, grad.norm=0.27294999
  5070: 3 [ 1060/ 1327], train_loss/perplexity = 6.42370605/616.2828369 secs/batch = 1.2550s, grad.norm=0.23974048


In [5]:
import importlib

In [6]:
importlib.reload(model)
model.TrainSentiModel(sess, saver, senti_train_graph, sentiReader, FLAGS.batch_size, FLAGS.batch_size)

data_X shape: (20, 31, 21)
data_Y shape: (20, 3)
sentX shape: Tensor("Placeholder_1:0", shape=(?, 31, 21), dtype=int32)
sent_y, shape: Tensor("Placeholder_2:0", shape=(?, 3), dtype=float32)
data_X shape: (20, 31, 21)
data_Y shape: (20, 3)
sentX shape: Tensor("Placeholder_1:0", shape=(?, 31, 21), dtype=int32)
sent_y, shape: Tensor("Placeholder_2:0", shape=(?, 3), dtype=float32)
data_X shape: (20, 31, 21)
data_Y shape: (20, 3)
sentX shape: Tensor("Placeholder_1:0", shape=(?, 31, 21), dtype=int32)
sent_y, shape: Tensor("Placeholder_2:0", shape=(?, 3), dtype=float32)
data_X shape: (20, 31, 21)
data_Y shape: (20, 3)
sentX shape: Tensor("Placeholder_1:0", shape=(?, 31, 21), dtype=int32)
sent_y, shape: Tensor("Placeholder_2:0", shape=(?, 3), dtype=float32)
data_X shape: (20, 31, 21)
data_Y shape: (20, 3)
sentX shape: Tensor("Placeholder_1:0", shape=(?, 31, 21), dtype=int32)
sent_y, shape: Tensor("Placeholder_2:0", shape=(?, 3), dtype=float32)
data_X shape: (20, 31, 21)
data_Y shape: (20, 3)
s

NameError: name 'logger' is not defined

In [6]:
importlib.reload(model)
model.TrainRDMModel(sess, rdm_train_graph, 0.7, 100)

Unknown char: ’
Word: that’s
Unknown char: ’
Word: said..he’s
Unknown char: ’
Word: they’re
Unknown char: "
Word: a"cleric
Unknown char: è
Word: ministère
Unknown char: ’
Word: l’intérieur
Unknown char: é
Word: l’intérieur
Unknown char: ’
Word: today’s
Unknown char: ’
Word: don’t
Unknown char: ’
Word: wouldn’t
Unknown char: "
Word: been"justifying
Unknown char: ░
Word: h░a░p░p░y
Unknown char: ░
Word: h░a░p░p░y
Unknown char: ░
Word: h░a░p░p░y
Unknown char: ░
Word: h░a░p░p░y
Unknown char: ░
Word: w░e░e░k
Unknown char: ░
Word: w░e░e░k
Unknown char: ░
Word: w░e░e░k
Unknown char: ░
Word: e░n░d
Unknown char: ░
Word: e░n░d
Unknown char: ░
Word: h░a░p░p░y
Unknown char: ░
Word: h░a░p░p░y
Unknown char: ░
Word: h░a░p░p░y
Unknown char: ░
Word: h░a░p░p░y
Unknown char: ░
Word: w░e░e░k
Unknown char: ░
Word: w░e░e░k
Unknown char: ░
Word: w░e░e░k
Unknown char: ░
Word: e░n░d
Unknown char: ░
Word: e░n░d
Unknown char: ░
Word: h░a░p░p░y
Unknown char: ░
Word: h░a░p░p░y
Unknown char: ░
Word: h░a░p░p░y
Unknow

NameError: name 'logger' is not defined

In [9]:
importlib.reload(model)
model.TrainCMModel(sess, rdm_train_graph, cm_train_graph, 0.3, 100)

in RL the begining
Unknown char: ’
Word: that’s
Unknown char: ’
Word: said..he’s
Unknown char: ’
Word: they’re
Unknown char: "
Word: a"cleric
Unknown char: è
Word: ministère
Unknown char: ’
Word: l’intérieur
Unknown char: é
Word: l’intérieur
Unknown char: ’
Word: today’s
Unknown char: ’
Word: don’t
Unknown char: ’
Word: wouldn’t
Unknown char: "
Word: been"justifying
Unknown char: ░
Word: h░a░p░p░y
Unknown char: ░
Word: h░a░p░p░y
Unknown char: ░
Word: h░a░p░p░y
Unknown char: ░
Word: h░a░p░p░y
Unknown char: ░
Word: w░e░e░k
Unknown char: ░
Word: w░e░e░k
Unknown char: ░
Word: w░e░e░k
Unknown char: ░
Word: e░n░d
Unknown char: ░
Word: e░n░d
Unknown char: ░
Word: h░a░p░p░y
Unknown char: ░
Word: h░a░p░p░y
Unknown char: ░
Word: h░a░p░p░y
Unknown char: ░
Word: h░a░p░p░y
Unknown char: ░
Word: w░e░e░k
Unknown char: ░
Word: w░e░e░k
Unknown char: ░
Word: w░e░e░k
Unknown char: ░
Word: e░n░d
Unknown char: ░
Word: e░n░d
Unknown char: ░
Word: h░a░p░p░y
Unknown char: ░
Word: h░a░p░p░y
Unknown char: ░
Wor

Unknown char: ề
Word: iều
Unknown char: ò
Word: hòa
Unknown char: ầ
Word: trần
Unknown char: =
Word: massmedia=junkfood
Unknown char: é
Word: gérard
Unknown char: 🐸
Word: french🐸frog
Unknown char: é
Word: défendre
Unknown char: é
Word: démocratie
Unknown char: ä
Word: gefährlich
Unknown char: ’
Word: that’s
Unknown char: é
Word: célèbrités
Unknown char: è
Word: célèbrités
Unknown char: é
Word: célèbrités
Unknown char: è
Word: ramènent
Unknown char: ê
Word: même
Unknown char: "
Word: issues."i
Unknown char: ’
Word: today’s
Unknown char: ’
Word: today’s
Unknown char: "
Word: unemplymt."they"deserve
Unknown char: "
Word: unemplymt."they"deserve
Unknown char: ’
Word: it’s
Unknown char: ’
Word: it’s
Unknown char: "
Word: i"m
Unknown char: é
Word: protégés
Unknown char: é
Word: protégés
Unknown char: ’
Word: they’re
Unknown char: …
Word: display…as
Unknown char: …
Word: staff…publish
Unknown char: …
Word: week…1m
Unknown char: "
Word: important"...more
Unknown char: "
Word: the"religion
Unkn

Unknown char: é
Word: spéisiúl
Unknown char: ú
Word: spéisiúl
Unknown char: ó
Word: fóill
Unknown char: í
Word: shuíomh
Unknown char: é
Word: insécurit
Unknown char: ’
Word: md’s
Unknown char: é
Word: astérix
Unknown char: é
Word: résistant
Unknown char: è
Word: très
Unknown char: á
Word: están
Unknown char: ó
Word: expresión
Unknown char: é
Word: también
Unknown char: í
Word: crítica
Unknown char: ’
Word: can’t
Unknown char: ’
Word: it’s
Unknown char: "
Word: article"how
Unknown char: =
Word: it=a
Unknown char: é
Word: l'amérique
Unknown char: "
Word: you're*...."as
Unknown char: "
Word: faith"when
Unknown char: ö
Word: angehörigen
Unknown char: ü
Word: mitgefühl
Unknown char: ü
Word: für
Unknown char: ’
Word: it’s
Unknown char: é
Word: aéreos
Unknown char: ’
Word: aren’t
Unknown char: ’
Word: that’s
Unknown char: ’
Word: gov’t
Unknown char: é
Word: norvégien
Unknown char: é
Word: dégouts
Unknown char: ’
Word: it’s
Unknown char: ’
Word: you’re
Unknown char: ’
Word: allan’s
Unknown cha

Unknown char: ’
Word: wolinski’s
Unknown char: ’
Word: don’t
Unknown char: ’
Word: wolinski’s
Unknown char: ’
Word: don’t
Unknown char: ’
Word: wolinski’s
Unknown char: ’
Word: don’t
Unknown char: ’
Word: wolinski’s
Unknown char: ’
Word: don’t
Unknown char: ’
Word: wolinski’s
Unknown char: ’
Word: don’t
Unknown char: é
Word: démocratie
Unknown char: é
Word: décidément
Unknown char: é
Word: décidément
Unknown char: ―
Word: conviction―just
Unknown char: ―
Word: conviction―just
Unknown char: ’
Word: don’t
Unknown char: ë
Word: dammartin-en-goël
Unknown char: ’
Word: you’re
Unknown char: ’
Word: it’s
Unknown char: ’
Word: they’re
Unknown char: ’
Word: aren’t
Unknown char: ’
Word: it’s
Unknown char: ’
Word: it’s
Unknown char: í
Word: víctimas
Unknown char: 😷
Word: daily😷😷😷your
Unknown char: 😷
Word: daily😷😷😷your
Unknown char: 😷
Word: daily😷😷😷your
Unknown char: =
Word: violations=dismissal
Unknown char: ü
Word: türkiye’de
Unknown char: ’
Word: türkiye’de
Unknown char: ş
Word: işi̇d
Unknown ch

Unknown char: é
Word: priorités
Unknown char: é
Word: déplac
Unknown char: â
Word: mankind.âll
Unknown char: =
Word: sharialaw=hamas=isis
Unknown char: =
Word: sharialaw=hamas=isis
Unknown char: =
Word: haram=hezbollah
Unknown char: "
Word: bold"?...trying
Unknown char: =
Word: lies=lies=lies
Unknown char: =
Word: lies=lies=lies
Unknown char: ’
Word: won’t
Unknown char: ’
Word: i’m
Unknown char: ’
Word: isn’t
Unknown char: ’
Word: us’s
Unknown char: ’
Word: isn’t
Unknown char: ’
Word: us’s
Unknown char: =
Word: suspects=more
Unknown char: é
Word: téméraire
Unknown char: é
Word: téméraire
Unknown char: ’
Word: i’d
Unknown char: ’
Word: wasn’t
Unknown char: è
Word: frère
Unknown char: é
Word: montée
Unknown char: í
Word: policía
Unknown char: í
Word: polícia
Unknown char: á
Word: austrália
Unknown char: é
Word: mantém
Unknown char: á
Word: vários
Unknown char: é
Word: reféns
Unknown char: ’
Word: won’t
Unknown char: ’
Word: he’s
Unknown char: ’
Word: he’s
Unknown char: "
Word: w/"reporte

Unknown char: =
Word: americans=full
Unknown char: ä
Word: väl
Unknown char: ä
Word: självklarhet
Unknown char: ’
Word: you’re
Unknown char: ’
Word: l’histoire
Unknown char: ’
Word: envoy’s
Unknown char: ’
Word: don’t
Unknown char: ’
Word: there’s
Unknown char: ’
Word: don’t
Unknown char: ’
Word: doesn’t
Unknown char: ’
Word: i’ve
Unknown char: ’
Word: that’s
Unknown char: ’
Word: don’t
Unknown char: ’
Word: i’d
Unknown char: ’
Word: it’s
Unknown char: "
Word: indication"of
Unknown char: ’
Word: i’m
Unknown char: ’
Word: i’m
Unknown char: ’
Word: people’s
Unknown char: ’
Word: let’s
Unknown char: ’
Word: i’ve
Unknown char: ’
Word: bush’s
Unknown char: ’
Word: bush’s
Unknown char: ’
Word: there’s
Unknown char: ’
Word: someone’s
Unknown char: ’
Word: it’s
Unknown char: ’
Word: there’s
Unknown char: ’
Word: someone’s
Unknown char: ’
Word: gun.it’s
Unknown char: ü
Word: sürüyor
Unknown char: ü
Word: sürüyor
Unknown char: é
Word: dénonce
Unknown char: "
Word: much"im
Unknown char: ’
Word: y

Unknown char: =
Word: murder=intentional
Unknown char: ’
Word: canada’s
Unknown char: ä
Word: jämföra
Unknown char: ö
Word: jämföra
Unknown char: å
Word: någon
Unknown char: ö
Word: förstår
Unknown char: å
Word: förstår
Unknown char: å
Word: sånt
Unknown char: ä
Word: här
Unknown char: ö
Word: för
Unknown char: ö
Word: förlorare
Unknown char: ä
Word: väldigt
Unknown char: é
Word: l'amérique
Unknown char: è
Word: derrière
Unknown char: ’
Word: i’d
Unknown char: ’
Word: there’s
Unknown char: ’
Word: doesn’t
Unknown char: ’
Word: it’s
Unknown char: ’
Word: don’t
Unknown char: ’
Word: can’t
Unknown char: ’
Word: you’re
Unknown char: ’
Word: i’m
Unknown char: ö
Word: möglich
Unknown char: ¨
Word: it¨s
Unknown char: â
Word: islâmico
Unknown char: ã
Word: são
Unknown char: ã
Word: estão
Unknown char: ’
Word: i’d
Unknown char: "
Word: war."once
Unknown char: "
Word: you"re
Unknown char: "
Word: in"jean
Unknown char: á
Word: barbárie
Unknown char: á
Word: más
Unknown char: ñ
Word: viñetas
Unkno

Unknown char: =
Word: dreadheads=savages
Unknown char: ’
Word: don’t
Unknown char: ’
Word: that’s
Unknown char: ’
Word: it’s
Unknown char: ’
Word: wouldn’t
Unknown char: ’
Word: i’m
Unknown char: ’
Word: doesn’t
Unknown char: ’
Word: i’m
Unknown char: ’
Word: it’s
Unknown char: ’
Word: i’m
Unknown char: ’
Word: i’m
Unknown char: ’
Word: it’s
Unknown char: ’
Word: that’s
Unknown char: â
Word: châtiment
Unknown char: é
Word: c'était
Unknown char: é
Word: véritable
Unknown char: è
Word: succès
Unknown char: =
Word: hispanicsasians=democrat
Unknown char: =
Word: christians=republicans
Unknown char: =
Word: christians=dems
Unknown char: ü
Word: mitgefühl
Unknown char: ö
Word: gehört
Unknown char: ö
Word: angehörigen
Unknown char: ̇
Word: i̇srailli
Unknown char: ü
Word: tümgeneral
Unknown char: ç
Word: koçavi
Unknown char: ü
Word: türkiye’de
Unknown char: ’
Word: türkiye’de
Unknown char: 
make: gun
Unknown char: 
rewardsun
Unknown char: 
puttingead
Unknown char: ’
Word: i’m
Unknown char: ’
W

KeyboardInterrupt: 